Using unitary gates of 1 qubit to reach a desired target state, the rewards are determined by element comparison

Import Packages

In [1]:
from numpy import *
import cmath
from random import randrange
import pandas as pd
import time
from IPython.display import clear_output

Define gates and states

In [2]:
a = 0.5
b = 0.5
one = array([[int(0)], [int(1)]])
zero = array([[int(1)], [int(0)]])
psi1 = (1/(sqrt(2)))*(zero + one)
psi1

array([[0.70710678],
       [0.70710678]])

In [3]:
X = array([[0 , 1], [1, 0]])
Z = array([[1,0],[0,-1]])
Iden = array([[1 , 0], [0, 1]])
Y = array([[0 , -1j], [1j, 0]])
H = (1/(sqrt(2)))*array([[1 , 1], [1, -1]])
S = array([[1 , 0], [0, 1j]])


In [4]:
right = Y.dot(Z.dot(X.dot(zero)))
right


array([[0.+1.j],
       [0.+0.j]])

Create list of actions and list of initial states

In [5]:
list2 = [Iden, X, Y, Z, H, S]

In [6]:
list1 = [zero, one, psi1]

In [7]:
def probability(guess, right):
    summ = 0
    for i in range(0,2):
        diff = (guess[i] - right[i])
        summ+= abs(diff)
    score = 1 - (summ)/2
    return(float(score))

First set up the dataframe with the correct columns and some initial states

In [25]:
Q_table = pd.DataFrame()

In [26]:
for i in range(0,len(list1)):
    new4 = list1[i]
    for m in range(0,len(list2)):
        gate = m
        new = (list2[m].dot(list1[i]))
        Q = pd.DataFrame({'state': str(new4), 'gate':gate, 'new':str(new) ,'Q':0}, index=[0])
        Q_table = Q_table.append(Q)


It's easier to call upon cells in string format rather than array format

In [27]:

Q_tablestr = Q_table.drop_duplicates()
Q_tablestr = Q_tablestr.reset_index(drop=True)

Set variables - these may need to be played about with more to find optimal values

In [28]:
table = Q_tablestr.copy()

In [29]:
alpha = 0.5 #how much we update Q for every new action
gamma = 0.9 #discount factor 

num_repeats = 100


In [30]:

for repeats in range(0,num_repeats):
    k = randrange(len(list1)) # choose a random state from list to start 
    psiold = list1[k]
    reward = 0 # reset award
    for t in range(0,5): #begin steps
        reward += 0.1 #add more penalty with each step
        
        sec = table[table['state']==str(psiold)] #choose the part of the dataframe that contains the current state
        
        gather = pd.DataFrame() #dataframe to collect all possible new states 
        for n in range(0,(len(sec))): #iterate through the rows that contain the current state
            for z in range(0,len(list2)): # find the action associated with the state in each row
                if(sec.iloc[n]['gate']== z):
                    apply = array(list2[z])
                    zed = z
                    
            psinew = apply.dot(psiold) #create new state by applying action
                
                
            news = table[table['state']==str(psinew)] #choose part of dataframe that contains the new state
            
            if len(news) == 0: #if the new state is not in the dataframe, insert it with all the possible actions
                for s in range(0,len(list2)):
                    lad = list2[s].dot(psinew)
                    news = pd.DataFrame({'state' : str(psinew), 'gate':s, 'new':str(lad) ,'Q':0}, index=[0])
                    table = table.append(news)
                    
            maxQ = max(news['Q']) #find the max Q value the new state
            gather = gather.append(news).drop_duplicates() #create a dataframe of all possible new states
            plus = 0 #reset award
            if (probability(psiold, right)) == 1: #insert a reward if the new state is the one we want
                plus = 1 
                    
            table.loc[(table.state == str(psiold)) & (table.gate == zed), 'Q'] += alpha*((plus - reward) + gamma*(maxQ) - sec.iloc[n]['Q'])
            #update the Q value of the old state based on the rewards given by the new state
        if plus == 1: #If we have reached the state we want we can stop
            print('goal reached')
            break
        else:
            top = gather.loc[(gather.Q == max(gather['Q']))] #collect the max Q value from all possible new states
            toparray =[] #sort these into an array with their associated action
            actarray =[]
            for o in range(0,len(top)):
                for p in range(0,len(list2)):
                    if (str((array(list2[p])).dot(psiold)) == (top.iloc[o]['state'])):
                        toparray.append((array(list2[p])).dot(psiold))
                        actarray.append(p)
        
            garray =[] #create an array of all possible new states with associated actions
            actg = []
            for a in range(0,len(top)):
                for b in range(0,len(list2)):
                    if (str((array(list2[b])).dot(psiold)) == (gather.iloc[a]['state'])):
                        garray.append((array(list2[b])).dot(psiold))
                        actg.append(b)
        
            q = randrange(100)
            if q > 50: #Create a percentage chance that the next state will be one with a max Q value (expoitation) or a state obtained from a random action (exploration)
                r = randrange(len(toparray))
                psiold = toparray[r]
                action = actarray[r]
            else:
                r = randrange(len(garray))
                psiold = garray[r]
                action = actg[r]
        
        #print(str(t) + 'th state is ' + str(psiold) + ' after action ' + str(action))
    print(repeats)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
goal reached
27
28
29
30
31
32
goal reached
33
34
goal reached
35
36
goal reached
37
38
39
40
41
42
goal reached
43
goal reached
44
goal reached
45
46
goal reached
47
48
49
50
goal reached
51
goal reached
52
goal reached
53
goal reached
54
55
56
57
goal reached
58
59
goal reached
60
61
goal reached
62
63
goal reached
64
65
goal reached
66
goal reached
67
goal reached
68
goal reached
69
goal reached
70
goal reached
71
72
goal reached
73
74
goal reached
75
goal reached
76
77
78
79
80
goal reached
81
goal reached
82
83
84
85
86
87
88
goal reached
89
goal reached
90
91
92
93
94
goal reached
95
96
97
98
99


Show the Q values from initial state

In [31]:
table[table['state'] == str(list2[0].dot(list2[0].dot(zero)))]

,state,gate,new,Q
0,[[1]\n [0]],0,[[1]\n [0]],2.885877
1,[[1]\n [0]],1,[[0]\n [1]],2.898784
2,[[1]\n [0]],2,[[0.+0.j]\n [0.+1.j]],3.292215
3,[[1]\n [0]],3,[[1]\n [0]],2.845312
4,[[1]\n [0]],4,[[0.70710678]\n [0.70710678]],2.168846
5,[[1]\n [0]],5,[[1.+0.j]\n [0.+0.j]],-0.717169


Show what actions are taken when following the max Q value for each state action pair

In [32]:
top1 = pd.DataFrame
for m in range(0,len(list1)):
    
    oppa = table.loc[(table.state == str(list1[m]))]
    top = oppa.loc[(oppa.Q == max(oppa['Q']))]
    for n in range(0,5):
        
        oppa1 = table.loc[(table.state == top['new'].values[n])]
        top1 = oppa1.loc[(oppa1.Q == max(oppa1['Q']))]
        
        top = top.append(top1)
        
top.head(30)
    

,state,gate,new,Q
16,[[0.70710678]\n [0.70710678]],4,[[1.]\n [0.]],2.621016
0,[[1.]\n [0.]],2,[[0.+0.j]\n [0.+1.j]],3.133975
0,[[0.+0.j]\n [0.+1.j]],1,[[0.+1.j]\n [0.+0.j]],3.882293
0,[[0.+1.j]\n [0.+0.j]],5,[[0.+1.j]\n [0.+0.j]],4.790862
0,[[0.+1.j]\n [0.+0.j]],5,[[0.+1.j]\n [0.+0.j]],4.790862
0,[[0.+1.j]\n [0.+0.j]],5,[[0.+1.j]\n [0.+0.j]],4.790862
